# Отладка Пайплайна Генерации STL

Этот notebook предназначен для пошаговой отладки всех этапов генерации STL модели.


In [1]:
import numpy as np
import sys
import os

# Добавляем текущую директорию в путь для импортов
current_dir = os.getcwd()
sys.path.insert(0, current_dir)

# Импорты модулей
from geometry.config import GeometryConfig
from geometry.coordinate_system import CoordinateSystem
from geometry.beveled_prism import BeveledPrism
from geometry.prism_layer import PrismLayer
from geometry.convex_solver import ConvexHullSolver
from transforms.cylindrical import CylindricalTransform
from export.stl_exporter import STLExporter
from layer_generator import LayerGenerator

print("✓ Все модули импортированы успешно")


✓ Все модули импортированы успешно


## Этап 1: Создание конфигурации

Проверяем, что параметры настроены правильно.


In [2]:
# Параметры (из legacy кода)
PI = np.pi
RADIUS = 4 * (1e2 * 1300 / 7500)
HEIGHT = 1 * 1e2
BEV_ANGLE = np.radians(30)

print(f"Параметры:")
print(f"  Radius: {RADIUS:.3f}")
print(f"  Height: {HEIGHT:.3f}")
print(f"  Bevel Angle: {np.degrees(BEV_ANGLE):.1f}°")

# Создаем конфигурацию
config = GeometryConfig(
    radius=RADIUS,
    height=HEIGHT,
    bev_angle=BEV_ANGLE,
    size_trick=1.1
)

print(f"\nКонфигурация: {config}")


Параметры:
  Radius: 69.333
  Height: 100.000
  Bevel Angle: 30.0°

Конфигурация: GeometryConfig(radius=69.33333333333333, height=100.0, bev_angle=0.524, size_trick=1.1)


## Этап 2: Тестирование одной призмы

Создаем и проверяем геометрию одной призмы.


In [3]:
# Создаем одну тестовую призму
center = np.array([0, 0, 0])
direction_height = np.array([0, 0, 1])
direction_radial = np.array([1, 0, 0])

prism = BeveledPrism(
    config=config,
    center=center,
    direction_height=direction_height,
    direction_radial=direction_radial
)

print(f"Призма создана: {prism}")
print(f"\nНачальные точки (begin_points): {len(prism.begin_points)}")
print(prism.begin_points)
print(f"\nКонечные точки (end_points): {len(prism.end_points)}")
print(prism.end_points)
print(f"\nВектора плоскостей: {len(prism.plane_vectors)}")


Призма создана: BeveledPrism(config=GeometryConfig(radius=69.33333333333333, height=100.0, bev_angle=0.524, size_trick=1.1), center=[0. 0. 0.], height_dir=[0. 0. 1.], radial_dir=[1. 0. 0.])

Начальные точки (begin_points): 8
[[ 0.00000000e+00  0.00000000e+00  5.00000000e+01]
 [ 6.00444280e+01  0.00000000e+00  0.00000000e+00]
 [ 3.00222140e+01  5.20000000e+01  0.00000000e+00]
 [-3.00222140e+01  5.20000000e+01  0.00000000e+00]
 [-6.00444280e+01  7.35332166e-15  0.00000000e+00]
 [-3.00222140e+01 -5.20000000e+01  0.00000000e+00]
 [ 3.00222140e+01 -5.20000000e+01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e+01]]

Конечные точки (end_points): 8
[[ 0.00000000e+00  0.00000000e+00  5.50000000e+01]
 [ 6.60488708e+01  0.00000000e+00  0.00000000e+00]
 [ 3.30244354e+01  5.72000000e+01 -3.00222140e+00]
 [-3.30244354e+01  5.72000000e+01  3.00222140e+00]
 [-6.60488708e+01  8.08865382e-15  0.00000000e+00]
 [-3.30244354e+01 -5.72000000e+01 -3.00222140e+00]
 [ 3.30244354e+01 -5.7200000

## Этап 3: Вычисление точек пересечения

Проверяем, находятся ли точки пересечения плоскостей.


In [4]:
# Создаем solver
solver = ConvexHullSolver()

# Условие соединения (из legacy кода)
def connect_func(point, center=np.array([0,0,0]), axes_vec=np.array([0,0,1])):
    point_rel = point - center
    axes_len = np.linalg.norm(axes_vec)
    axes_ord = axes_vec / axes_len
    rad_dist = np.sqrt(point_rel[0]**2 + point_rel[1]**2 + point_rel[2]**2 - (np.dot(point_rel, axes_ord)**2))
    
    kostyl_border = 1
    if point[2]**2 > HEIGHT**2 or rad_dist > (RADIUS + 0.5 * HEIGHT / np.tan(BEV_ANGLE)) * kostyl_border:
        return True
    return True

# Находим точки пересечения
intersection_points = solver.find_plane_intersections(
    vectors=prism.plane_vectors,
    points=prism.begin_points,
    connect_condition=connect_func
)

print(f"Найдено точек пересечения: {len(intersection_points)}")
if len(intersection_points) > 0:
    print(f"\nПример точек пересечения:")
    print(intersection_points[:5])
else:
    print("\n⚠️ ПРОБЛЕМА: Не найдено ни одной точки пересечения!")


Найдено точек пересечения: 12

Пример точек пересечения:
[[ 60.044428    63.53418013  50.        ]
 [ 50.          69.33333333  50.        ]
 [-60.044428     5.79915321  50.        ]
 [-60.044428   -63.53418013  50.        ]
 [-50.         -69.33333333  50.        ]]


## Этап 4: Построение Convex Hull

Строим convex hull из точек пересечения.


In [5]:
# Строим convex hull
if len(intersection_points) >= 4:
    faces = solver.build_convex_hull(intersection_points)
    print(f"Построено граней: {len(faces)}")
    
    if len(faces) > 0:
        print(f"\nПример грани (треугольник):")
        print(faces[0])
    else:
        print("\n⚠️ ПРОБЛЕМА: Convex hull не построен!")
else:
    print(f"\n⚠️ ПРОБЛЕМА: Недостаточно точек для convex hull (нужно минимум 4, есть {len(intersection_points)})")
    faces = []


Построено граней: 20

Пример грани (треугольник):
[array([-50.        , -69.33333333,  50.        ]), array([-60.044428  ,  -5.79915321, -50.        ]), array([ 50.        , -69.33333333, -50.        ])]


## Этап 5: Полный тест через LayerGenerator ->OK

Тестируем весь пайплайн с маленьким слоем.


In [6]:
# Тестируем полный пайплайн через LayerGenerator
print("=== ПОЛНЫЙ ТЕСТ ЧЕРЕЗ LayerGenerator ===")

generator = LayerGenerator(config)

# Генерируем маленький слой
print("\n1. Генерация слоя...")
layer = generator.generate_layer(x_num=2, y_num=2)
print(f"   ✓ Создано призм: {len(layer)}")

# Вычисляем грани
print("\n2. Вычисление граней...")
faces = generator.compute_faces()
total_faces = sum(len(f) for f in faces)
print(f"   ✓ Всего граней: {total_faces}")

# Экспортируем
if total_faces > 0:
    print("\n3. Экспорт в STL...")
    generator.export_stl("test_debug.stl", format="ascii")
    print("   ✓ Экспорт завершен: test_debug.stl")
else:
    print("\n⚠️ Нет граней для экспорта!")


=== ПОЛНЫЙ ТЕСТ ЧЕРЕЗ LayerGenerator ===

1. Генерация слоя...
   ✓ Создано призм: 6

2. Вычисление граней...
   ✓ Всего граней: 120

3. Экспорт в STL...
ASCII-STL written: test_debug.stl
  • Unique vertices: 72
  • Triangles: 120
   ✓ Экспорт завершен: test_debug.stl


## Этап 6: Тест с большим слоем (как в main.py)

Тестируем слой 11x11, как в main.py.


In [7]:
# Тест с большим слоем (как в main.py)
print("=== ТЕСТ С БОЛЬШИМ СЛОЕМ 11x11 ===")

# Создаем новый генератор
generator_large = LayerGenerator(config, bend_radius=30 * 1e6)

# Генерируем большой слой
print("\n1. Генерация слоя 11x11...")
layer_large = generator_large.generate_layer(x_num=11, y_num=11)
print(f"   ✓ Создано призм: {len(layer_large)}")

# Вычисляем грани
print("\n2. Вычисление граней...")
faces_large = generator_large.compute_faces()
total_faces_large = sum(len(f) for f in faces_large)
print(f"   ✓ Всего граней: {total_faces_large}")

# Экспортируем
if total_faces_large > 0:
    print("\n3. Экспорт в STL...")
    generator_large.export_stl("test_large.stl", format="ascii")
    print("   ✓ Экспорт завершен: test_large.stl")
else:
    print("\n⚠️ Нет граней для экспорта!")


=== ТЕСТ С БОЛЬШИМ СЛОЕМ 11x11 ===

1. Генерация слоя 11x11...
   ✓ Создано призм: 142

2. Вычисление граней...
   ✓ Всего граней: 2840

3. Экспорт в STL...
ASCII-STL written: test_large.stl
  • Unique vertices: 1704
  • Triangles: 2840
   ✓ Экспорт завершен: test_large.stl


## Этап 7: Тест generate_complete_layer (как в main.py)

Тестируем метод, который используется в main.py.


In [ ]:
# Тест generate_complete_layer (как в main.py)
print("=== ТЕСТ generate_complete_layer ===")

# Создаем генератор точно как в main.py
generator_complete = LayerGenerator(config, bend_radius=30 * 1e6)

# Используем тот же метод, что и в main.py
print("\n1. Вызов generate_complete_layer...")
layer_complete = generator_complete.generate_complete_layer(
    x_num=11,
    y_num=11,
    bend_radius=30 * 1e6,
    output_filename="test_complete.stl",
    solid_name="bf8_sls",
    format="ascii"
)

print(f"   ✓ Создано призм: {len(layer_complete)}")
print(f"   ✓ Граней: {sum(len(f) for f in generator_complete.faces) if generator_complete.faces else 0}")

# Проверяем, что файл создался
import os
if os.path.exists("test_complete.stl"):
    size = os.path.getsize("test_complete.stl")
    print(f"   ✓ Файл создан: test_complete.stl (размер: {size} байт)")
else:
    print("   ⚠️ Файл не создан!")


## Этап 8: Детальная диагностика одной призмы

Проверяем, что именно происходит с геометрией одной призмы.


In [ ]:
# Детальная диагностика одной призмы
print("=== ДЕТАЛЬНАЯ ДИАГНОСТИКА ОДНОЙ ПРИЗМЫ ===")

# Создаем одну призму
center = np.array([0, 0, 0])
direction_height = np.array([0, 0, 1])
direction_radial = np.array([1, 0, 0])

prism = BeveledPrism(
    config=config,
    center=center,
    direction_height=direction_height,
    direction_radial=direction_radial
)

print(f"\n1. Геометрия призмы:")
print(f"   Center: {prism.center}")
print(f"   Config: radius={prism.config.radius}, height={prism.config.height}")

print(f"\n2. Begin points ({len(prism.begin_points)}):")
for i, pt in enumerate(prism.begin_points):
    print(f"   [{i}]: {pt}")

print(f"\n3. End points ({len(prism.end_points)}):")
for i, pt in enumerate(prism.end_points):
    print(f"   [{i}]: {pt}")

print(f"\n4. Plane vectors ({len(prism.plane_vectors)}):")
for i, vec in enumerate(prism.plane_vectors):
    norm = np.linalg.norm(vec)
    print(f"   [{i}]: {vec} (norm={norm:.3f})")

# Проверяем, все ли векторы ненулевые
zero_vectors = sum(1 for vec in prism.plane_vectors if np.linalg.norm(vec) < 1e-10)
print(f"\n5. Нулевых векторов: {zero_vectors}")

# Тестируем вычисление пересечений вручную
print(f"\n6. Тест вычисления пересечений:")
test_intersections = solver.find_plane_intersections(
    vectors=prism.plane_vectors,
    points=prism.begin_points,
    connect_condition=lambda p: True  # Без фильтрации
)
print(f"   Найдено пересечений (без фильтра): {len(test_intersections)}")

if len(test_intersections) > 0:
    print(f"   Примеры точек:")
    for i, pt in enumerate(test_intersections[:5]):
        print(f"     [{i}]: {pt}")
else:
    print("   ⚠️ НЕТ ПЕРЕСЕЧЕНИЙ!")


## Этап 9: Тест влияния size_trick

Проверяем, как size_trick влияет на генерацию STL.


In [ ]:
# Тест влияния size_trick на генерацию STL
print("=== ТЕСТ ВЛИЯНИЯ size_trick ===")

# Тестируем разные значения size_trick
test_values = [0.5, 0.8, 1.0, 1.01, 1.1, 1.21, 1.5, 2.0]

for size_trick in test_values:
    print(f"\n--- Тест с size_trick = {size_trick} ---")
    
    # Создаем конфигурацию с новым size_trick
    test_config = GeometryConfig(
        radius=RADIUS,
        height=HEIGHT,
        bev_angle=BEV_ANGLE,
        size_trick=size_trick
    )
    
    # Создаем генератор
    generator = LayerGenerator(test_config)
    
    # Генерируем маленький слой
    layer = generator.generate_layer(x_num=2, y_num=2)
    
    # Вычисляем грани
    faces = generator.compute_faces()
    total_faces = sum(len(f) for f in faces)
    
    print(f"   Призм: {len(layer)}, Граней: {total_faces}")
    
    if total_faces > 0:
        print(f"   ✅ УСПЕХ!")
        
        # Проверяем геометрию первой призмы
        first_prism = layer.prisms[0]
        print(f"   Begin points: {len(first_prism.begin_points)}")
        print(f"   End points: {len(first_prism.end_points)}")
        print(f"   Plane vectors: {len(first_prism.plane_vectors)}")
        
        # Проверяем, что векторы ненулевые
        zero_vectors = sum(1 for vec in first_prism.plane_vectors if np.linalg.norm(vec) < 1e-10)
        print(f"   Нулевых векторов: {zero_vectors}")
        
        if zero_vectors > 0:
            print(f"   ⚠️ Есть нулевые векторы!")
    else:
        print(f"   ❌ Не работает")


# Отладка STL Layer Generator Pipeline

Этот ноутбук предназначен для пошаговой отладки всего пайплайна генерации STL слоев.
